In [1]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om

import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, coo_matrix, identity, vstack
from scipy.sparse.linalg import spsolve, lsqr
from scipy.spatial import Delaunay, Voronoi

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline
# Tests of Openmesh
# https://gitlab.vci.rwth-aachen.de:9000/OpenMesh/openmesh-python/tree/master/tests


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh, vertex_color=True)
def show_mesh_o3d(plys):
    o3d.visualization.draw_geometries(plys)
def write_mesh_o3d(path, mesh):
    o3d.io.write_triangle_mesh(path, mesh)
def read_mesh_o3d(mesh_fp):
    return o3d.io.read_triangle_mesh(mesh_fp)
def read_mesh_trimesh(path_in_models):
    return trimesh.load(os.path.join('..', 'models', path_in_models))
def get_vertices_from_trimesh(mesh):
    return np.asarray(mesh.vertices)
def get_vertice_normals_from_trimesh(mesh):
    return np.asarray(mesh.vertex_normals)


In [3]:

### Calculate Voronoi Poles
INFINITY = 1e9
EPS = 1e-9
def calculate_voronoi_poles(mesh):
    
    vertices = get_vertices_from_trimesh(mesh)
    vertice_normals = get_vertice_normals_from_trimesh(mesh)

    print("Calculating Voronoi Diagram.")
    vor = Voronoi(vertices)
    vor_centers = vor.vertices
    cells = vor.regions
    cell_indices = vor.point_region

    vertices_total = vertices.shape[0]
    voronoi_poles = np.zeros(vertices.shape)
    print("Calculating the Voronoi Pole for each vertex.")
    for vi in tqdm(range(vertices_total)):
        vor_cell = cells[cell_indices[vi]]
        vertice = vertices[vi]
        vertice_normal = vertice_normals[vi]
        max_neg_proj = INFINITY
        voronoi_pole = None
        for vci in vor_cell:
            vor_center = vor_centers[vci]
            if vci == -1:
                continue
            proj = np.dot(vor_center - vertice, vertice_normal)
            if proj < max_neg_proj:
                max_neg_proj = proj
                voronoi_pole = vor_center
        voronoi_poles[vi] = voronoi_pole

    return voronoi_poles


In [4]:

### Calculate Cotangent Laplace Operator
def trimesh_Generate_Area_List(mesh):

    area_list = np.zeros((len(mesh.vertices)))
    face_count = mesh.faces.shape[0]
    face_area = np.asarray(mesh.area_faces)
    faces = np.asarray(mesh.faces)
    for i in range(face_count):
        area_list[faces[i]] += face_area[i] / 3
    
    area_list /= np.mean(area_list)
    return area_list


def trimesh_Generate_Laplace_matrix(mesh):
    
    print('Constructing Laplace Matrix.')
    
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = trimesh_Generate_Area_List(mesh)
    face_angles = np.asarray(mesh.face_angles)
    faces = np.asarray(mesh.faces)
    print(len(faces))
    
    
    # with tqdm(total=face_count) as tbar:
    for face, angles in zip(faces, face_angles):
        for i in range(3):
            current_angle = angles[i]
            v0_index, v1_index, v2_index = face[vertices_face_indexs[i]]
            delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
            if delta > EPS:
                laplace_dict[(v1_index, v1_index)] = laplace_dict.get((v1_index, v1_index), 0) - delta
                laplace_dict[(v2_index, v2_index)] = laplace_dict.get((v2_index, v2_index), 0) - delta
                laplace_dict[(v1_index, v2_index)] = laplace_dict.get((v1_index, v2_index), 0) + delta
                laplace_dict[(v2_index, v1_index)] = laplace_dict.get((v2_index, v1_index), 0) + delta
        # tbar.update(1)

    # Construct CSR Matrix
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())
    print(delta,np.mean(values))
    coo = coo_matrix((values, (rows, cols)), shape=(len(mesh.vertices), len(mesh.vertices)))
    
    csr = coo.tocsr()

    return csr


In [5]:

### Solve Equation
def construct_Equation(wL, wH, wM, vertices, L, voronoi_poles):
    vn = vertices.shape[0]
    WH_L = wH * identity(vn)
    WM_L = wM * identity(vn)
    A = vstack((wL * L, WH_L, WM_L))
    b = np.vstack((np.zeros((vn, 3)), wH * vertices, wM * voronoi_poles))
    return A, b


In [7]:
### Laplace Smooth
flatten = lambda l: [item for sublist in l for item in sublist]

def laplacian_smoothing(mesh, smooth_factor=1.0, iterations=1):
   
    for _ in range(iterations):
        n = mesh.vertices.shape[0]
        vertex_degs = [len(neighbors) for neighbors in mesh.vertex_neighbors]
        sparse_mat_rows = [[row_idx] * (deg + 1) for row_idx, deg in enumerate(vertex_degs)]
        sparse_mat_cols = [neighbors + [row_idx] for row_idx, neighbors in enumerate(mesh.vertex_neighbors)]

        
        sparse_mat_weights = [[-1.0] * deg + [deg * 1.0] for deg in vertex_degs]   

        sparse_mat_rows = flatten(sparse_mat_rows)
        sparse_mat_cols = flatten(sparse_mat_cols) 

        sparse_mat_weights = [float(u) / row_weights[-1]
                                    for row_weights in sparse_mat_weights
                                    for u in row_weights]
        L =coo_matrix((sparse_mat_weights, (sparse_mat_rows, sparse_mat_cols)), shape=(n , n))
        L = L.tocsr()  # Compress the sparse matrix..
        v = np.array(mesh.vertices)
        diff = L * v
        mesh.vertices = v - smooth_factor * diff

In [29]:
def get_shared_face_index(mesh, v1, v2):
    # Get the faces containing vertex v1
    faces_with_v1 = np.where(np.any(mesh.faces == v1, axis=1))[0]
    # Get the faces containing vertex v2
    faces_with_v2 = np.where(np.any(mesh.faces == v2, axis=1))[0]
    # Find the intersection of the two arrays to get the shared face index
    shared_face_index = np.intersect1d(faces_with_v1, faces_with_v2)
    # print(shared_face_index)
    return shared_face_index

In [53]:
#         v0 ---------- v2
#        /        +     /
#      /      vn       /
#    /     +          /   
#  v1 ---------------v3
def reduce_Angles(mesh):
    # face_angles = mesh.face_angles
    faces = np.asarray(mesh.faces)
    f_is_del = np.zeros((len(mesh.faces)),dtype=bool) # f is deleted or not
    f_to_add = np.asarray(mesh.faces) # all v indexs. Note that here it only adds points and dont remove any.
    face_angles = np.asarray(mesh.face_angles)

    vertices = np.asarray(mesh.vertices)
    v_to_add = np.asarray(mesh.vertices) # positions
    v_count = len(mesh.vertices)
    with tqdm(total=len(f_to_add)) as tbar:
        for face_index, face in enumerate(faces):
            if ~f_is_del[face_index] and max(face_angles[face_index])>2.4: # 2.1
                
                # get v0,v1,v2
                v0 = face[np.argmax(face_angles[face_index])] # obtuse_vertex
                v1,v2 = face[face!=v0] # v1,v2 are indexes.                

                # add vertex vn
                v_count +=1
                v_to_add = np.vstack((v_to_add,[(vertices[v1]+vertices[v2])/2]))
                # print(v_to_add)

                # get indexs of faces to be deleted
                shared_faces_index = get_shared_face_index(mesh, v1, v2)
                f_is_del[shared_faces_index]= True

                # # get v3 from the faces
                v_vertices = np.asarray(np.unique(faces[shared_faces_index]))
                v3 = int(np.setdiff1d(v_vertices, (v0,v1,v2))) # ----may cause problems----                

                # # add faces
                vn = v_count-1
                f_to_add = np.vstack((f_to_add,[v0,v1,vn]))
                f_to_add = np.vstack((f_to_add,[v0,vn,v2])) # the order matters! if go v0v2vn, not work
                f_to_add = np.vstack((f_to_add,[vn,v1,v3])) # why
                f_to_add = np.vstack((f_to_add,[v2,vn,v3]))                
                f_is_del = np.append((f_is_del),[False,False,False,False])
                # break
            tbar.update(1)
    faces_new = f_to_add[~f_is_del]
    print(f_is_del)
    mesh.vertices = v_to_add
    mesh.faces = faces_new
    return mesh
    

In [31]:

name = 'cube-copy'
mesh = read_mesh_trimesh("test/"+name+'.off')
# print(mesh.edges_unique)
mesh = reduce_Angles(mesh)
# print(mesh.faces)
# print(mesh.faces) 
write_mesh_o3d('../models/result/test_cut_obtuse.obj', mesh.as_open3d)



100%|██████████| 12/12 [00:00<?, ?it/s]

[False False False False False False  True  True False False  True  True
 False False False False False False False False]


In [50]:
(mesh.face_angles).max()
mesh = reduce_Angles(mesh)
(mesh.face_angles).max() # ? biandale  wcao

100%|██████████| 50382/50382 [00:05<00:00, 9661.22it/s] 

[False False False ... False False False]


3.1353898159464295

In [229]:
# Introduction and Criticism

# to make sure obtuse is smaller, the threshold make be bigger than 120 degree
# x/2 + 180 - x < x ==> x > 120

#     A
#    /
#   /
#  B-------------D-------------C
# BAC is X, if ACD exist and ADC < BAC(obtuse smaller), then x/2+180-x<x

# 110 will not gurantee this!! (110 + 69 + 1） will make a bigger obtuse!

In [47]:
### Iteration
it = 1
wL_list = [20]#[20,10,1.5,1,0.5,0.05,0.005,0.0005,0.00001]
wH_list = [0.5]#[20,10,1.5,1,0.5,0.05,0.005,0.0005,0.00001]
wM_list = [1.5]#[20,10,1.5,1,0.5,0.05,0.005,0.0005,0.00001]
name = 'armadillo'
new_mesh = name + '_0_smooth.obj'
# new_mesh = 'asculpt.obj'
mesh = read_mesh_trimesh(name+'.obj')
voronoi_poles = calculate_voronoi_poles(mesh)
for wL in wL_list:
    for wH in wH_list:
        for wM in wM_list:
            for i in range(it):
                mesh = read_mesh_trimesh(name+'.obj')
                print("processing"+str(wL)+str(wH)+str(wM))

                laplace_csr = trimesh_Generate_Laplace_matrix(mesh)
                # if i > 0:
                #     laplace_csr *= -10 * np.power(10,i)
                # print(np.mean(laplace_csr))
                vertices = get_vertices_from_trimesh(mesh)
                A, b = construct_Equation(wL, wH, wM, vertices, laplace_csr, voronoi_poles)

                new_vertices0 = lsqr(A, b[:, 0])
                new_vertices1 = lsqr(A, b[:, 1])
                new_vertices2 = lsqr(A, b[:, 2])

                new_vertices = np.zeros(vertices.shape)
                new_vertices[:, 0] = new_vertices0[0]
                new_vertices[:, 1] = new_vertices1[0]
                new_vertices[:, 2] = new_vertices2[0]

                mesh.vertices = new_vertices
                
                # save_index = int(i/3)
                # mesh = mesh.smoothed()
                # mesh = mesh.subdivide_to_size(average_edge_length(mesh))
                # laplacian_smoothing(mesh,smooth_factor = 1.0)
                # mesh = subdivide_based_on_angle(mesh, 0.9)
                new_mesh = name +str(i)+'_%d_smooth.obj' 
                write_mesh_o3d('../models/result/' + new_mesh, mesh.as_open3d)
                # mesh = read_mesh_trimesh('result/' + new_mesh)
                # laplacian_smoothing(mesh,smooth_factor = 1.0)
                # new_mesh = name +str(wL)+str(wH)+str(wM) +'_%d_smooth.obj' % (save_index + 1)
                # smoothed_mesh = '%d_smooth.obj' % (save_index + 1)
                # write_mesh_o3d('../models/result/' + smoothed_mesh, mesh.as_open3d)
                # print("Done one time-----------------------------")
            
(mesh.face_angles).max()


Calculating Voronoi Diagram.
Calculating the Voronoi Pole for each vertex.


100%|██████████| 25193/25193 [00:01<00:00, 20742.52it/s]


processing200.51.5
Constructing Laplace Matrix.
50382
2.482933512073437 0.0


3.130051856485212

In [ ]:
# if threshold: 1.8 3.1300 -> 3.1353
# if 2.1 : 

In [ ]:
def collapse_mesh(mesh):
    vertices = mesh.vertices
    faces = mesh.faces